In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/4.csv").iloc[0:len(postulaciones2)//2], ignore_index=True)
print(len(postulaciones2))
postulaciones2.head()

9906801


,idaviso,idpostulante,sepostulo
0,1112257047,ow2xPaj,0
1,1111920714,LNKXqJB,0
2,1112346945,8MBPqYD,0
3,1112345547,ek4P2Wr,0
4,1112237522,NpqMD5,0


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16450211


In [9]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
1299229,1112270219,NzdzaaV,0,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
13265979,1112295959,ameLW5,1,MASC,28.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,NaN
3501161,1112320214,5mk5XwG,0,MASC,34.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
4331643,1112364685,GNZamN9,1,MASC,27.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
12901084,1112443807,ZOXB88,1,FEM,31.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,2.0
13745080,1112267771,4rlkjdq,0,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
13975736,1112141113,xkd8ALK,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,NaN
8730884,1112422710,JBrDrze,1,MASC,34.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,1.0
1621661,1112350139,JBr2zEE,0,FEM,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
7199819,1112319451,mzdWqOE,0,FEM,22.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,NaN


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
10239710,1112388955,2zkEQ50,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,2.0
13712402,1112416238,YjQv04w,1,MASC,52.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,True,False,True,5.0
7849531,1112329035,xk9XLw6,0,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
11293151,1112290068,6rPKpLv,0,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,False,False,False,0.0
14738563,1112336931,DrXYrWL,0,MASC,55.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.0
4077856,1112447442,QNlex0z,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,1.0
10479437,1112213148,VNeJJwR,1,MASC,33.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
12425479,1112423655,EBJG09,0,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
8577667,1112353008,ow22vK0,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.0
12450565,1112203791,8MZrpDR,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  14560399 Test:  766337


In [12]:
gc.collect()

156

In [13]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=4096,max_depth=13,n_estimators=3000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

76.14456825130458


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV9.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV9.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
26708,88748,1112467018,JBrNDJO,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
60054,8472,1111780418,YZ8xP,FEM,44.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,True,False,True,0.0
79180,26692,1112343364,ZrkwVZ,MASC,53.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.0
25181,84206,1112466532,NwzmNl,FEM,28.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,0.0
3358,42005,1112422210,JBxxKrE,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,1.0
47806,2788,1111301974,5Zqbjk,FEM,39.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
55696,5487,1111558613,ZrKoYE,MASC,39.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.0
29634,78737,1112464571,owE3G0e,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
68986,37820,1112414480,exYjVo,MASC,52.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
38158,15068,1112115684,6RZX5R,MASC,46.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,True,False,0.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.82517391, 0.17482609],
       [0.6740859 , 0.3259141 ],
       [0.68893989, 0.31106011],
       ...,
       [0.89813383, 0.10186617],
       [0.70970782, 0.29029218],
       [0.97744906, 0.02255094]])

In [19]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [20]:
prediccion['sepostulo']=y_final2

In [21]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.174826
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.325914
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.311060
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.087415
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.276297


In [22]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.174826
1,1,0.325914
2,2,0.311060
3,3,0.087415
4,4,0.276297


In [24]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion23.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.5342532727094954